In [16]:
# Exerceise 1 :  Implementing a matrix factorisation using gradient descent

# Exercise 1.1 : Implement gradient based factorisation

from typing import Tuple
import torch

def sgd_factorise (A: torch.Tensor , rank : int , num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    U = torch.rand(list(A.size())[0],rank)
    V = torch.rand(list(A.size())[1],rank)
    for epoch in range(0, num_epochs):
        for r in range(0, list(A.size())[0]):# Check this. Does it start with 0 or 1 becuase the the end value is not inclusive in the loop count.
            for c in range(1, list(A.size())[1]):
                e = A[r][c] - (U[r,:] * V[c,:].t())
                U[r,:] = U[r,:] + lr * e * V[c,:]
                V[c,:] = V[c,:] + lr * e * U[r,:]
    return U,V

# Exercise 1.2 : Factorise and compute reconstruction error

A = torch.tensor([[0.3374,0.6005,0.1735] , [3.3359,0.0492,1.8374] , [2.9407,0.5301,2.2620]])
U_cap , V_cap = sgd_factorise(A , 2)

#print("Tensor U cap is : " , U_cap)
#print("Tensor V cap is : " , V_cap)

# Computing reconstruction error here. Reconstruction is given by U_cap @ V_cap.t().
# The reconstruction error is give by || A - U_cap@V_capt() ||
recon_loss = torch.nn.functional.mse_loss(A, U_cap@V_cap.t(), reduction='sum')
print("The reconstruction error is : " , recon_loss)

The reconstruction error is :  tensor(37.6393)
Reconstruction loss before setting last singular value to zero :  tensor(33.9710)
Reconstruction error after setting last singular value to zero :  tensor(33.9210)


In [ ]:
# Exercise 2 : Comparison of results to truncated SVD

u , s , v = torch.svd(A)
#print(u@torch.diag(s)@v)
new_recon_loss = torch.nn.functional.mse_loss(A, u@torch.diag(s)@v , reduction='sum')
print("Reconstruction loss before setting last singular value to zero : " , new_recon_loss)

# Setting the last singular value to 0
s[2] = 0
#print(s)
final_recon_loss = torch.nn.functional.mse_loss(A, u@torch.diag(s)@v , reduction='sum')
print("Reconstruction error after setting last singular value to zero : " , final_recon_loss)

In [18]:
# Exercise 3 : Matrix Completion

# Exercise 3.1 : Implementation of masked factorisation

def sgd_factorise_masked(A: torch.Tensor, M:torch.Tensor, rank:int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    U = torch.rand(list(A.size())[0] , rank)
    V = torch.rand(list(A.size())[1] , rank)
    for epoch in range(1,num_epochs):
        for r in range(1, list(A.size())[0]):
            for c in range(1, list(A.size())[1]):
                if(M[r][c] != 0):
                    e = A[r][c] - U[r]@V[c].t()
                    U[r] = U[r] + lr*e*V[c]
                    V[c] = V[c] + lr*e*U[r]
    return U,V
                    

# Exercise 3.2 : Reconstruct a Matrix

A = torch.tensor([[0.3374,0.6005,0.1735] , [0,0.0492,1.8374] , [2.9407,0,2.2620]])
M = torch.tensor([[1,1,1],[0,1,1],[1,0,1]])
new_U , new_V = sgd_factorise_masked(A,M,2)
#print(new_U)
#print(new_V)
            
    
reconstructed_matrix = new_U @ new_V.t()
print("The estimated values are: ")
print("Row 2 , column 1 : " , reconstructed_matrix[1][0])
print("Row 3 , column 2 : " , reconstructed_matrix[2][1])

print("The original matrix is : " , A)


The estimated values are: 
Row 2 , column 1 :  tensor(0.5140)
Row 3 , column 2 :  tensor(0.1094)
The original matrix is :  tensor([[0.3374, 0.6005, 0.1735],
        [0.0000, 0.0492, 1.8374],
        [2.9407, 0.0000, 2.2620]])


In [ ]:
# Doubts : For the rank loop, does it start with 1 or does it start with 0. confirm this becuase the number of 
    # times the loop will run will vary
    